Celem projektu jest przygotowanie sysytemu rekomendacji piosenek na podstawie ich tekstów. Jedną z ważniejszych rzeczy jest przygotowanie zestawu danych, które umożliwią przeprowadzenie badań. W tym przypadku podstawą danych jest dataset Genius Song Lyrics with Information (https://www.kaggle.com/datasets/carlosgdcj/genius-song-lyrics-with-language-information/). Jednakże jest to bardzo duży dataset, dlatego niezbędne jest przeprowadzenie preprocessingu danych. Należy zacząć od pobrania biblioteki pandas.

In [ ]:
%pip install pandas

Po zainstalowaniu biblioteki, należy ją zimportować i następnie po pobraniu datasetu ze strony, wczytać go do programu. Poniżej znajduje się pierwsze 5 wierszy pliku, a także infomacja o kształcie(wymiarach tabeli) i lista z nazwami kolumn.


In [3]:
import pandas as pd
data = pd.read_csv("song_lyrics.csv")
data.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [5]:
data.shape

(5134856, 11)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5134856 entries, 0 to 5134855
Data columns (total 11 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   title          object
 1   tag            object
 2   artist         object
 3   year           int64 
 4   views          int64 
 5   features       object
 6   lyrics         object
 7   id             int64 
 8   language_cld3  object
 9   language_ft    object
 10  language       object
dtypes: int64(3), object(8)
memory usage: 430.9+ MB


Jak można zauważyć jest to bardzo duży dataset, ponieważ posiada aż 5134856 wierszy. Znajdują się w nim piosenki w wielu językach. Do projketu zostaną wykorzystane tylko piosenki w języku angielskim, które odfiltrujemy z datasetu. W związku z tym, kolumny ze specyfikacją języka też są niepotrzebne.

In [10]:
data.drop(data[data['language'] != 'en'].index, inplace=True)

In [12]:
data.drop(["language","language_cld3","language_ft"], axis=1, inplace=True)
data.head()

,title,tag,artist,year,views,features,lyrics,id
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3374198 entries, 0 to 5134855
Data columns (total 9 columns):
 #   Column    Dtype   
---  ------    -----   
 0   title     object  
 1   tag       object  
 2   artist    object  
 3   year      int64   
 4   views     int64   
 5   features  object  
 6   lyrics    object  
 7   id        int64   
 8   decade    category
dtypes: category(1), int64(3), object(5)
memory usage: 234.9+ MB


Kolejnym krokiem jest podzielenie piosenek ze względu na ich rok powstania. 

In [43]:
data['decade'] = pd.cut(data['year'], bins=[1919, 1929, 1939, 1949, 1959, 1969, 1979, 1989, 1999, 2009, 2019, 2029, 2039],
                        labels=['1920s', '1930s', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s', '2020s', 'Other'])

grouped_data = data.groupby('decade')

decade_counts = grouped_data.size()

print(decade_counts)

decade
1920s       2530
1930s       1820
1940s       1624
1950s      10227
1960s      42208
1970s      65177
1980s      97164
1990s     222928
2000s     452208
2010s    1727308
2020s     731567
Other          0
dtype: int64


C:\Users\zofia\AppData\Local\Temp\ipykernel_27276\1350829036.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = data.groupby('decade')


Z wykresów wynika, że większość danych piosenek pochodzi z dekad 2000 i 2010.

In [42]:
size_1970_2000 = data[(data['year'] >= 1970) & (data['year'] < 2000)].shape[0]

print("Size for 1950-2000:", size_1970_2000)


Size for 1950-2000: 385269


W projekcie zbadamy zestaw piosenek z dekad 1970-2000, ze względu na większe zainteresowanie tym okresem muzycznym przez współautorów pracy, a także odpowiednią wielkość datasetu.

In [50]:
#zapisac dane dla piosenek 1950-2000 do pliku csv
d = data[(data['year'] >= 1970) & (data['year'] < 2000)]
d.to_csv('data.csv', index=False)

In [51]:
d.head()

,title,tag,artist,year,views,features,lyrics,id,decade
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,1990s
8,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}",[Verse 1]\nAy yo you wonder who I are\nI guzzl...,10,1990s
12,DEvils,rap,JAY-Z,1996,504959,{},[Produced by DJ Premier]\n\n[Hook Samples: Sno...,14,1990s
16,Its Hot Some Like It Hot,rap,JAY-Z,1999,103549,{},"[Produced by Timbaland]\n\n[Verse 1]\nYo, show...",18,1990s
18,Its Like That,rap,JAY-Z,1998,37692,"{""Kid Capri""}","[Intro: Jay-Z, Kid Capri & Both]\nYeah, uh-huh...",123,1990s
